In [3]:
import numpy as np
import pandas as pd
import theano
import theano.tensor as T

import lasagne

theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


In [7]:
path="data/"
data=pd.read_csv(path+"input_train.csv")
labels=pd.read_csv(path+"challenge_output_data_training_file_sleep_stages_classification.csv", sep=";")

In [8]:
def build_cnn(input_var=None):
    network = lasagne.layers.InputLayer(shape=(None, 1, 3750, 1),
                                        input_var=input_var)
    network = lasagne.layers.Conv2DLayer(
        network, num_filters=16, filter_size=(25, 1),
        nonlinearity=lasagne.nonlinearities.rectify,
        W=lasagne.init.GlorotUniform())
    network = lasagne.layers.MaxPool2DLayer(network, pool_size=(5, 1))
    #network = lasagne.layers.Conv2DLayer(
    #    network, num_filters=32, filter_size=(10, 1),
    #    nonlinearity=lasagne.nonlinearities.rectify,
    #    W=lasagne.init.GlorotUniform())
    #network = lasagne.layers.MaxPool2DLayer(network, pool_size=(5, 1))
    network = lasagne.layers.DenseLayer(
        lasagne.layers.dropout(network, p=.5),
        num_units=1000,
        nonlinearity=lasagne.nonlinearities.rectify)
    network = lasagne.layers.DenseLayer(
        lasagne.layers.dropout(network, p=.5),
        num_units=5,
        nonlinearity=lasagne.nonlinearities.softmax)

    return network

In [34]:
# Load the dataset
from sklearn.cross_validation import train_test_split
X_temp, X_test, y_temp, y_test = train_test_split(data, labels["TARGET"], test_size=0.10, random_state=42)
X_train, y_train, X_val, y_val = train_test_split(X_temp, y_temp, test_size=0.10, random_state=42)
# Prepare Theano variables for inputs and targets
input_var = T.tensor4('inputs')
target_var = T.ivector('targets')
# Create neural network model
network = build_cnn(input_var)

In [10]:
prediction = lasagne.layers.get_output(network)
loss = lasagne.objectives.multiclass_hinge_loss(prediction, target_var)
loss = loss.mean()

In [12]:
params = lasagne.layers.get_all_params(network, trainable=True)
updates = lasagne.updates.nesterov_momentum(
        loss, params, learning_rate=0.01, momentum=0.9)

In [13]:
test_prediction = lasagne.layers.get_output(network, deterministic=True)
test_loss = lasagne.objectives.multiclass_hinge_loss(test_prediction,
                                                        target_var)
test_loss = test_loss.mean()

In [14]:
test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), target_var),
                  dtype=theano.config.floatX)

In [18]:
from tempfile import mkstemp

In [19]:
tempfile.mkstemp(dir="../tmp")

(43, '/data/home/sofia.calcagno/tmp/tmp2D96HD')

In [25]:
import os
os.environ["THEANO_FLAGS"] = "blas.ldflags=-llapack -lblas -lgfortran"

In [26]:
train_fn =theano.function([input_var, target_var], loss, updates=updates)

ImportError: ('The following error happened while compiling the node', CorrMM{(0, 0), (1, 1)}(inputs, Subtensor{::, ::, ::int64, ::int64}.0), '\n', '/data/home/sofia.calcagno/.theano/compiledir_Linux-3.10-el7.x86_64-x86_64-with-centos-7.0.1406-Core-x86_64-2.7.5-64/tmpb_cnuv/7bbe82694c11837e1e668153b2b582f0.so: undefined symbol: dgemm_', '[CorrMM{(0, 0), (1, 1)}(inputs, <TensorType(float64, 4D)>)]')

In [ ]:
val_fn = theano.function([input_var, target_var], [test_loss, test_acc])

In [ ]:
for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train, 500, shuffle=True):
        inputs, targets = batch
        train_err += train_fn(inputs, targets)
        train_batches += 1

    # And a full pass over the validation data:
    val_err = 0
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_val, y_val, 500, shuffle=False):
        inputs, targets = batch
        err, acc = val_fn(inputs, targets)
        val_err += err
        val_acc += acc
        val_batches += 1

    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))
    print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
    print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
    print("  validation accuracy:\t\t{:.2f} %".format(
        val_acc / val_batches * 100))

In [ ]:
predicted_values = test_prediction.flatten()
print "test loss:", test_loss